In [17]:
#Author: Noam Karasso

import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
import time

#getting all languages and shortcuts from wiki for fun ;)
url = "https://meta.wikimedia.org/wiki/Template:List_of_language_names_ordered_by_code"
html = requests.get(url).content.decode('utf-8')
soup = BeautifulSoup(html,'html.parser')
langs = soup.find_all('td')
lang_dict = {}
for i in range(len(langs) // 6):
  lang_dict[langs[1*i*6].get_text().strip()] = langs[1*i*6+1].get_text().strip()

surnames_dict = {}
names_dict = {}
def get_names(page,length,new_dict):
  p = page[length].findAll('a')
  if 'Given' in p[0] or 'Disambiguation' in p[0]:
    get_names(page,length+1, new_dict)
  else:
    for name in p:
      new_dict.update({name.get_text() : []})
      #print(name.get_text())
    if length < len(page)-1:
      get_names(page,length+1,new_dict)

#functions that remove some unwanted pages from the names dictionary
def filter_trash():
  names_dict.pop("Template:R from given name")
  names_dict.pop("List of most popular given names")
  names_dict.pop("Given name")
  names_dict.pop("Given name stubs")
  names_dict.pop("Disambiguation pages with given-name-holder lists")

def filter_trash2():
  surnames_dict.pop("Surname stubs")
  surnames_dict.pop("Surname templates")
  surnames_dict.pop("Yujiulü clan")
  surnames_dict.pop("Surnames by language")
  surnames_dict.pop("Surnames by culture")
  surnames_dict.pop("Lists of people by surname")
  surnames_dict.pop("Lists of surnames")
  surnames_dict.pop("Compound surnames")
  surnames_dict.pop("Ethnonymic surnames")
  surnames_dict.pop("Gendered surnames")
  surnames_dict.pop("Occupational surnames")
  surnames_dict.pop("Patronymic surnames")
  surnames_dict.pop("Redirects from surnames")
  surnames_dict.pop("Shi'ite surnames")
  surnames_dict.pop("Surnames from given names")
  surnames_dict.pop("Surnames from nicknames")
  surnames_dict.pop("Surnames from ornamental names")
  surnames_dict.pop("Surnames from status names")
  surnames_dict.pop("Toponymic surnames")
  surnames_dict.pop("Theophoric names")


#iterate through all the pages by taking the "next page" url for names
l = "w/index.php?title=Category:Given_names"
while 'until' not in l:
  url = f"https://en.wikipedia.org/{l}"
  html = requests.get(url).content.decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')

  pages = soup.findAll('div',{"class": "mw-category-group"})
  get_names(pages,len(pages)-3,names_dict)

  a_content = soup.findAll('a',{"title": "Category:Given names"})
  l = a_content[len(a_content)-1]['href']

filter_trash()
print(len(names_dict))

#iterate through all the pages by taking the "next page" url for surnames
l = "w/index.php?title=Category:Surnames"
while 'until' not in l:
  url = f"https://en.wikipedia.org/{l}"
  html = requests.get(url).content.decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')

  pages = soup.findAll('div',{"class": "mw-category-group"})
  get_names(pages,0,surnames_dict)

  a_content = soup.findAll('a',{"title": "Category:Surnames"})
  l = a_content[len(a_content)-1]['href']
  #print(l)

filter_trash2()
#merge both dicts to avoid redundancy
names_dict.update(surnames_dict)


19452


# function that extracts the page id and gets all info from it

In [18]:
def detailsByID(name):
  # Construct the API URL
  api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={name}&prop=pageprops&format=json'
  time.sleep(0.01)#might help with the connection error
  # Make the API request and convert the response to JSON
  response = requests.get(api_url)
  #if response.status_code == 200:
  response = response.json()
  # Extract the page ID from the JSON response and get all data
  try:
    id = next(iter(response['query']['pages'].values()))['pageprops']['wikibase_item']
    #print(name)
    url = f"https://www.wikidata.org/wiki/{id}"
    html = requests.get(url).content.decode('utf-8')
    soup = BeautifulSoup(html,'html.parser')
    label = soup.find_all('span',{'class':"wikibase-labelview-text"})
    label = label[0].get_text()
    #print(label)
    description = soup.find_all('span',{"class":"wikibase-descriptionview-text"})
    description = description[0].get_text()
    entries = soup.find_all('ul',{"class":"wikibase-sitelinklistview-listview"})
    entries = entries[0].find_all('span',{"class":"wikibase-sitelinkview-page"})
    for e in entries:
      lang = e['lang']
      language = lang_dict[lang]
      entry = e.get_text()
      result_tuples = (label,id,description,language,lang,entry)
      result.append(result_tuples)
  except KeyError:
    pass

def tryConnection(name):
  try:
      detailsByID(name)
  except ConnectionResetError:#THIS DOES NOT WORK
    tryConnection(name)

names_list = list(names_dict.keys())

# we'll split the input to 6 parts as a workaround to deal with the connection errors

In [20]:
result = []

for i in range(20000):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names1.csv', index=False, encoding='utf-8-sig')
#files.download('names1.csv')

In [ ]:
result = []

for i in range(20000,40000):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names2.csv', index=False, encoding='utf-8-sig')
#files.download('names2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result = []

for i in range(40000,60000):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names3.csv', index=False, encoding='utf-8-sig')
#files.download('names3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result = []

for i in range(60000,80000):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names4.csv', index=False, encoding='utf-8-sig')
#files.download('names4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result = []

for i in range(80000,100000):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names5.csv', index=False, encoding='utf-8-sig')
#files.download('names5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result = []

for i in range(100000,len(names_list)):
  name = names_list[i]
  tryConnection(name)

results_df = pd.DataFrame(result, columns=['Label', 'WikiData ID', 'English Description', 'Language', 'Wiki Short Lang', 'Entry'])
results_df.to_csv('names6.csv', index=False, encoding='utf-8-sig')
#files.download('names6.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# After we have all the csv files we need we merge them into one and download

In [ ]:
df1 = pd.read_csv('/content/names1.csv')
df2 = pd.read_csv('/content/names2.csv')
df3 = pd.read_csv('/content/names3.csv')
df4 = pd.read_csv('/content/names4.csv')
df5 = pd.read_csv('/content/names5.csv')
df6 = pd.read_csv('/content/names6.csv')

df = pd.concat([df1,df2,df3,df4,df5,df6])
df.to_csv('names.csv', index=False, encoding='utf-8-sig')
files.download('names.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>